# Geospatial Analytics with Python
<hr style="border:1px solid black"> </hr>

# 
Laian Abussaud

## Contents
<hr style="border:0.5px solid black"> </hr>


1. Definitions

2. Applications 

3. Comparison

4. Python

5. Demo

## Definitions
<hr style="border:0.5px solid black"> </hr>


#### What is GIS?

Geographic Informaton System, a system that allows you to visualize, analyze, and store data to understand <ins>spatial</ins> relationships, patterns, and trends.


#### What is Geospatial data?

Geospatial data is data about objects, events, or phenomena that has a <ins>location</ins>. Usually stored as coordinates (latitude, longtitude).


<img src="lat_long.png" style="width:750px;height:600px"/>

<img src="data_types_2.png" style="width:600px;height:600px"/>

#### What is Geospatial analysis?

A mix of data science, advanced analysis, and maps!


#### It helps us answer *Where* questions

- Where is?
- Where should we place?
- Where will the hurricane hit?
- Where should we place advertisements?

## Applications
<hr style="border:0.5px solid black"> </hr>


> "GIS is widely used in organizations of all sizes and in almost every industry."


Some GIS analysis applications include:
- Architecture/Urban planning
- Map Navigation
- Transportation Planning
- Accident/Hot Spot Analysis
- Aviation Applications
- Environmental Impact Analysis
- Agricultural Applications
- Astronomy Applications
- Disaster Management and Mitigation
- Water Management
- Banking (ATM Machines)
- etc.

## Comparision
<hr style="border:0.5px solid black"> </hr>


**Traditional Desktop GIS Software:**

Tools:
   - ArcGIS by Esri
   - QGIS
   
Pros:
   - Very good for cartography
   - Does not require coding
   - If you prefer GUI's
    
Cons:
   - Not great for big data
   - Not great for documentation
   - Not great for statistical analysis

**Geospatial Data Science Softwares:**
    
Tools:
   - Python
   - R
   
Pros:
   - Great for big data
   - Great for documentation
   - Great for statistical analysis
    
Cons:
   - Not good for cartography
   - Requires coding
   - Steeper learning curve

## Python
<hr style="border:0.5px solid black"> </hr>


### Base map packages:

- XYZ tiles, to provide the base map layer
- ex. **Folium** →  a python wrapper for Leaflet, to plot interactive maps

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[24.78, 46.75], zoom_start=5, control_scale=True)
m

In [ ]:
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)
m

### Street Network packages:

- **NetworkX** →  Aric Hagberg, Pieter Swart, and Dan Schult	
- **Pandana** →  Fletcher Foti
- **OSMnx** →  Geoff Boeing

### Data manipulation packages:

- **Pandas**, for high-performance, easy-to-use data structures and data analysis tools
- **GeoPandas**, for Geographically-enabled Pandas, depends on <ins>Shapely</ins> and <ins>Fiona</ins>, it adds columns in Pandas for spatial operations

### OpenStreetMap (OSM):

<a href="https://www.openstreetmap.org/export#map=6/23.080/49.735">www.openstreetmap.org</a>

<a href="https://wiki.openstreetmap.org/wiki/Key:amenity">Key:amenity - OpenStreetMap Wiki</a>

<img src="OSM.jpg" style="width:1000px;height:500px"/>

<img src="osm_2.png" style="width:1000px;height:500px"/>

- Not just a map, it is a **Database**
- Open-source collaborative project
- Attributes saved as key-value pairs
- Amenity key

<img src="osm_3.png" style="width:1000px;height:500px"/>

## Demo
<hr style="border:0.5px solid black"> </hr>

#### Spatial Question: How accessible/walkable are the cafes in my district?

#### 0. Import python libraries:


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandana
from pandana.loaders import osm

#### 1. Specify OSM bounding box, amenity type and distance

In [ ]:
bbox = [26.2841, 50.1995, 26.3366, 50.2454] # lat-long bounding box, [Bottom, Left, Top, Right]
amenity = 'cafe' # accessibility to this type of amenity
distance = 1500 # max distance in meters

#### 2. Download points of interest (POIs)

In [ ]:
pois = osm.node_query(bbox[0], bbox[1], bbox[2], bbox[3], tags='"amenity"="{}"'.format(amenity))
pois.tail()

#### 3. Download street network data from OSM

In [ ]:
# query the OSM API for the street network within the specified bbox
# need to install osmnet to download 
network = osm.pdna_network_from_bbox(bbox[0], bbox[1], bbox[2], bbox[3])
len(network.node_ids)

#### 4. Process the network data then compute accessibility

In [ ]:
# identify nodes that are connected to fewer than some threshold of other nodes within a given distance
# do nothing with this for now, but see full example in other notebook for more
lcn = network.low_connectivity_nodes(impedance=1000, count=10, imp_name='distance')
# precomputes the range queries (the reachable nodes within this maximum distance)
# so, as long as you use a smaller distance, cached results will be used
network.precompute(distance + 1)
# initialize the underlying C++ points-of-interest engine
network.init_pois(num_categories=1, max_dist=distance, max_pois=7)
# initialize a category for this amenity with the locations specified by the lon and lat columns
network.set_pois(category='my_amenity', x_col=pois['lon'], y_col=pois['lat'])
# search for the n nearest amenities to each node in the network
access = network.nearest_pois(distance=distance, category='my_amenity', num_pois=7)
# each df cell represents the network distance from the node to each of the n POIs
access.head()

#### 5. Plot accessibility

In [ ]:
# keyword arguments to pass for the matplotlib figure
bbox_aspect_ratio = (bbox[2] - bbox[0]) / (bbox[3] - bbox[1])
fig_kwargs = {'facecolor':'w', 
              'figsize':(10, 10 * bbox_aspect_ratio)}

# keyword arguments to pass for scatter plots
plot_kwargs = {'s':5, 
               'alpha':0.9, 
               'cmap':'viridis_r', 
               'edgecolor':'none'}

# plot the distance to the nth nearest amenity
n = 1
bmap, fig, ax = network.plot(access[n], bbox=bbox, plot_kwargs=plot_kwargs, fig_kwargs=fig_kwargs)
ax.set_axis_bgcolor('k')
ax.set_title('Walking distance (m) to nearest {} around Al-Khobar, Saudi Arabia'.format(amenity), fontsize=15)
plt.show()

In [ ]:
import osmnx as ox

G = ox.graph_from_bbox(26.3366, 26.2841, 50.2454, 50.1995, network_type='drive')
ox.plot_graph(G, figsize=(6, 12))

## Thank you for listening!

### Any questions?


## RISE settings

In [ ]:
from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path
path = Path.home() / ".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
tmp = cm.update(
        "rise",
        {
            "theme": "serif",
            "transition": "fade",
            "start_slideshow_at": "beginning",
            "autolaunch": True,
            "width": "100%",
            "height": "100%",
            "header": "",
            "footer":"",
            "scroll": True,
            "enable_chalkboard": True,
            "slideNumber": True,
            "center": True,
            "controlsLayout": "edges",
            "slideNumber": True,
            "hash": True,
        }
    )